In [35]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

file_path = 'dados_sem_nulos.csv'
df = pd.read_csv(file_path)

print("Dados carregados com sucesso!")
print(f"O dataset tem {df.shape[0]} linhas e {df.shape[1]} colunas.\n")

numerical_cols = [
    'C008',
    'consumo_semanal_frutas',
    'consumo_semanal_verdura_legume',
    'minutos_atv_fis_semana',
    'IMC'
]

categorical_cols = [col for col in df.columns if col not in numerical_cols]

print("Gerando boxplots para colunas numéricas...")

output_dir_num = 'boxplots_numericos'
if not os.path.exists(output_dir_num):
    os.makedirs(output_dir_num)

for col in numerical_cols:
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=df[col])
    plt.title(f'Boxplot para {col}', fontsize=15)
    plt.xlabel(col, fontsize=12)

    plt.savefig(os.path.join(output_dir_num, f'boxplot_{col}.png'))
    plt.close()

print(f"Boxplots salvos na pasta: '{output_dir_num}'")

print("\n--- Análise de Frequência para Colunas Categóricas ---")

for col in categorical_cols:
    print(f"\nContagem de valores para a coluna: '{col}'")

    freq_percent = df[col].value_counts(normalize=True) * 100

    freq_abs = df[col].value_counts()

    freq_df = pd.DataFrame({
        'Contagem': freq_abs,
        'Porcentagem (%)': freq_percent.round(2)
    })
    
    print(freq_df)

print("\n--- Análise de Outliers Concluída ---")

Dados carregados com sucesso!
O dataset tem 88736 linhas e 31 colunas.

Gerando boxplots para colunas numéricas...
Boxplots salvos na pasta: 'boxplots_numericos'

--- Análise de Frequência para Colunas Categóricas ---

Contagem de valores para a coluna: 'P01101'
        Contagem  Porcentagem (%)
P01101                           
3          20027            22.57
2          17542            19.77
7          12241            13.79
4          10716            12.08
1           9345            10.53
5           8481             9.56
0           7348             8.28
6           3036             3.42

Contagem de valores para a coluna: 'P013'
      Contagem  Porcentagem (%)
P013                           
2        23610            26.61
3        21975            24.76
1        10628            11.98
4        10277            11.58
5         7424             8.37
0         6580             7.42
7         5588             6.30
6         2654             2.99

Contagem de valores para a coluna

In [36]:
#tratando atributos com winsoriazação
df['N001'] = np.where(df['N001'] >= 4, 4, df['N001'])
df['P050'] = np.where(df['P050'] == 2, 1, df['P050'])

#tratando atributos com agrupamento 
soma = df['P02001'] + df['P02002']

conditions = [
    (soma == 0), 
    (soma.between(1, 2)),
    (soma.between(3, 4)),
    (soma >= 5)
]

# 0='Nada', 1='Pouco', 2='Moderado', 3='Muito'
labels = [0, 1, 2, 3]

df['bebidas_acucar_dias_semana'] = np.select(conditions, labels, default=np.nan)

#tratando atributos com binnig
def converter_consumo_semanal(col):
    bins = [-np.inf, 0, 2, 4, np.inf]

    #0 dias, 1-2 dias, 3-4 dias, 5+ dias
    labels = [0, 1, 2, 3]

    df[col] = pd.cut(df[col], 
                                bins=bins, 
                                labels=labels, 
                                right=True)

cols = ['P01101', 'P013', 'P015', 'P023', 'P00901',
        'P018', 'P02501', 'P02602']

for col in cols:
    converter_consumo_semanal(col)

for col in cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col])

#1 = alto, 2 = moderado, 3 = baixo consumo de sal
mapa = {
    1: 1,
    2: 1,
    3: 2,
    4: 3,
    5: 3
}

df['P02601'] = df['P02601'].map(mapa)

#essa coluna ainda ficaria com uma classe muito escassa, então estou
#juntando ambas
df['P02602'] = np.where(df['P02602'] == 3, 2, df['P02602'])

#removendo atributos com classe muito predominante
df = df.drop(columns=['infarto', 'angina', 'insuficiencia_cardiaca',
                  'arritmia', 'outra_doenca_cardiaca', 'Q124',
                  'P02001', 'P02002'])
#como esses atributos a maioria é doença cardiaca (que já está no dataset)
#e o outro não é tão relevante, não vou remover as instâncias

In [37]:
categorical_cols.append('bebidas_acucar_dias_semana')

itens_para_remover = [
    'infarto', 
    'angina', 
    'insuficiencia_cardiaca',
    'arritmia', 
    'outra_doenca_cardiaca', 
    'Q124',
    'P02002',
    'P02001'
]

for item in itens_para_remover:
    try:
        categorical_cols.remove(item)
    except ValueError:
        pass 


for col in categorical_cols:
    print(f"\nContagem de valores para a coluna: '{col}'")

    freq_percent = df[col].value_counts(normalize=True) * 100

    freq_abs = df[col].value_counts()

    freq_df = pd.DataFrame({
        'Contagem': freq_abs,
        'Porcentagem (%)': freq_percent.round(2)
    })
    
    print(freq_df)

print("\n--- Análise de Outliers Concluída ---")


Contagem de valores para a coluna: 'P01101'
        Contagem  Porcentagem (%)
P01101                           
2          30743            34.65
1          26887            30.30
3          23758            26.77
0           7348             8.28

Contagem de valores para a coluna: 'P013'
      Contagem  Porcentagem (%)
P013                           
1        34238            38.58
2        32252            36.35
3        15666            17.65
0         6580             7.42

Contagem de valores para a coluna: 'P015'
      Contagem  Porcentagem (%)
P015                           
0        41000            46.20
1        35906            40.46
2         7766             8.75
3         4064             4.58

Contagem de valores para a coluna: 'P023'
      Contagem  Porcentagem (%)
P023                           
3        40210            45.31
0        26974            30.40
1        11980            13.50
2         9572            10.79

Contagem de valores para a coluna: 'P00901'
 

In [38]:
# 1. Definir os limites (bins) para 4 categorias
bins = [
    -1,                   # Limite inferior
    0,                    # Categoria 1: Sedentário (apenas o 0)
    149,                  # Categoria 2: Insuficientemente Ativo (1 a 149)
    299,                  # Categoria 3: Ativo (150 a 299)
    np.inf                # Categoria 4: Muito Ativo (300+)
]

# 2. Definir os rótulos (labels)
labels = [
    0, #'sedentario'
    1, #'insuficientemente_ativo'
    2, #'ativo_recomendado'
    3, #'muito_ativo'
]

# 3. Criar a nova coluna categórica
# Use a coluna ORIGINAL de minutos, não a '_yeo'
df['atividade_fisica_categoria'] = pd.cut(
    df['minutos_atv_fis_semana'], 
    bins=bins, 
    labels=labels, 
    right=True 
)
df['atividade_fisica_categoria'] = pd.to_numeric(df['atividade_fisica_categoria'])

print("Categorias criadas e sua distribuição:")
print(df['atividade_fisica_categoria'].value_counts().sort_index())

bins_imc = [
    0, 18.5, 24.9, 29.9, 34.9, 39.9, np.inf
]
# 'Abaixo do Peso', 'Peso Normal', 'Sobrepeso', 
# 'Obesidade Grau I', 'Obesidade Grau II', 'Obesidade Grau III'
labels_imc = [1, 2, 3, 4, 5, 6]

# 3. Criar a nova coluna categórica
df['IMC_Categoria'] = pd.cut(df['IMC'], bins=bins_imc, labels=labels_imc, right=False)
df['IMC_Categoria'] = pd.to_numeric(df['IMC_Categoria'])

# 4. Verificar o resultado
print("Contagem de pacientes por Categoria de IMC (Método Recomendado):")
print(df['IMC_Categoria'].value_counts().sort_index())

Categorias criadas e sua distribuição:
atividade_fisica_categoria
0    54354
1    11732
2    11319
3    11331
Name: count, dtype: int64
Contagem de pacientes por Categoria de IMC (Método Recomendado):
IMC_Categoria
1     2133
2    34357
3    33533
4    13764
5     3737
6     1212
Name: count, dtype: int64


In [39]:
df = df.drop(columns=['consumo_semanal_verdura_legume', 
                      'consumo_semanal_frutas',  'IMC', 'minutos_atv_fis_semana'])
#vou remover essas colunas porque o tratamento delas é muito desbalanceado
#já que só conta quantas vezes por dia a pessoa consome a partir de 5 dias por s
df

,P01101,P013,P015,P023,P00901,P018,P02501,P02602,P02601,P027,...,Q06306,Q060,N001,N010,N011,C008,C006,bebidas_acucar_dias_semana,atividade_fisica_categoria,IMC_Categoria
0,3,1,1,3,3,1,2,0,2,1,...,2,2,1,1,2,30,1,3.0,2,3
1,3,1,1,3,2,3,3,0,2,1,...,2,2,2,1,1,80,2,0.0,0,2
2,0,3,0,0,3,3,1,0,3,1,...,2,1,2,1,1,56,2,1.0,1,3
3,2,2,1,0,3,3,1,0,2,1,...,2,2,2,1,4,34,2,0.0,0,2
4,2,2,0,1,2,2,0,0,2,1,...,2,2,2,1,1,40,2,0.0,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88731,1,2,1,0,2,1,0,0,2,1,...,2,2,2,4,4,30,2,2.0,0,5
88732,1,1,1,2,2,2,0,0,2,3,...,2,2,2,1,1,39,1,1.0,3,3
88733,1,2,1,2,2,1,1,0,2,3,...,2,2,2,1,1,34,1,2.0,0,4
88734,2,2,1,2,3,3,0,0,2,1,...,2,1,1,1,1,51,2,1.0,2,3


In [ ]:
dicionario = pd.read_excel("dicionarios/dicionario_PNS_microdados_2019_23062023.xlsx", header=1)
dic_limpo = dicionario.drop(dicionario.columns[[0, 1, 3, 5, 6]], axis=1)
dic_limpo = dic_limpo.rename(columns={
    dic_limpo.columns[0]: "codigo",
    dic_limpo.columns[1]: "descricao"
})
dic_limpo.dropna(subset=['codigo'], inplace=True)
dic_limpo.reset_index(drop=True, inplace=True)

tipos_das_colunas = df.dtypes

dic_completo = pd.DataFrame({
    'codigo': tipos_das_colunas.index,
    'tipo': tipos_das_colunas.values
})

dic_completo['tipo'] = dic_completo['tipo'].astype(str)

dic_completo = pd.merge(
    dic_completo,
    dic_limpo[['codigo', 'descricao']],
    on='codigo',
    how='left'
)

dic_completo['descricao'] = dic_completo['descricao'].fillna("")

dic_completo = dic_completo[['codigo', 'descricao', 'tipo']]

print("Dicionário de dados completo (com colunas novas) e tipos:")
display(dic_completo)

Dicionário de dados completo (com colunas novas) e tipos:


,codigo,descricao,tipo
0,P01101,Em quantos dias da semana o(a) Sr(a) costuma c...,int64
1,P013,Em quantos dias da semana o(a) Sr(a) costuma c...,int64
2,P015,Em quantos dias da semana o(a) Sr(a) costuma c...,int64
3,P023,Em quantos dias da semana o(a) Sr(a) costuma t...,int64
4,P00901,"Em quantos dias da semana, o(a) Sr(a) costuma ...",int64
5,P018,Em quantos dias da semana o(a) Sr(a) costuma c...,int64
6,P02501,Em quantos dias da semana o(a) Sr(a) costuma c...,int64
7,P02602,Em quantos dias da semana o(a) Sr(a) costuma s...,int64
8,P02601,Considerando a comida preparada na hora e os a...,int64
9,P027,Com que frequência o(a) Sr(a) costuma consumir...,int64


In [42]:
mapa_renomear = {
    # --- Hábitos Alimentares ---
    'P01101': 'consumo_carne_vermelha',
    'P013': 'consumo_frango',
    'P015': 'consumo_peixe',
    'P023': 'consumo_leite',
    'P00901': 'consumo_verdura_legume',
    'P018': 'consumo_frutas',
    'P02501': 'consumo_doces',
    'P02602': 'consumo_substitui_almoco',
    'P02601': 'percepcao_consumo_sal',
    
    # --- Outros Hábitos de Vida ---
    'P027': 'freq_bebida_alcoolica',
    'P050': 'fumante_atual',
    
    # --- Diagnósticos Médicos (Saúde) ---
    'Q00201': 'diag_hipertensao',
    'Q06306': 'diag_doenca_coracao',
    'Q060': 'diag_colesterol_alto',
    
    # --- Saúde Geral e Bem-estar ---
    'N001': 'autoavaliacao_saude',
    'N010': 'freq_problemas_sono',
    'N011': 'freq_cansaco',
    
    # --- Demográficos ---
    'C008': 'idade',
    'C006': 'sexo'
}

# --- Como usar ---

# 1. Renomear as colunas no seu DataFrame principal
df = df.rename(columns=mapa_renomear)

# 2. (Importante) Atualizar o seu 'dic_completo' para refletir os novos nomes
#    Isso garante que seu dicionário de dados permaneça correto.
dic_completo['codigo'] = dic_completo['codigo'].replace(mapa_renomear)

# 3. (Opcional) Salvar este novo dicionário renomeado
dic_completo.to_csv('dicionarios/dicionario_renomeado.csv', index=False, sep="|")

df.to_csv("dados_sem_outliers.csv", index=False)

# Verificar o resultado
print("DataFrame com colunas renomeadas:")
display(df.head())

print("\nDicionário de dados atualizado:")
display(dic_completo)

DataFrame com colunas renomeadas:


,consumo_carne_vermelha,consumo_frango,consumo_peixe,consumo_leite,consumo_verdura_legume,consumo_frutas,consumo_doces,consumo_substitui_almoco,percepcao_consumo_sal,freq_bebida_alcoolica,...,diag_doenca_coracao,diag_colesterol_alto,autoavaliacao_saude,freq_problemas_sono,freq_cansaco,idade,sexo,bebidas_acucar_dias_semana,atividade_fisica_categoria,IMC_Categoria
0,3,1,1,3,3,1,2,0,2,1,...,2,2,1,1,2,30,1,3.0,2,3
1,3,1,1,3,2,3,3,0,2,1,...,2,2,2,1,1,80,2,0.0,0,2
2,0,3,0,0,3,3,1,0,3,1,...,2,1,2,1,1,56,2,1.0,1,3
3,2,2,1,0,3,3,1,0,2,1,...,2,2,2,1,4,34,2,0.0,0,2
4,2,2,0,1,2,2,0,0,2,1,...,2,2,2,1,1,40,2,0.0,3,5



Dicionário de dados atualizado:


,codigo,descricao,tipo
0,consumo_carne_vermelha,Em quantos dias da semana o(a) Sr(a) costuma c...,int64
1,consumo_frango,Em quantos dias da semana o(a) Sr(a) costuma c...,int64
2,consumo_peixe,Em quantos dias da semana o(a) Sr(a) costuma c...,int64
3,consumo_leite,Em quantos dias da semana o(a) Sr(a) costuma t...,int64
4,consumo_verdura_legume,"Em quantos dias da semana, o(a) Sr(a) costuma ...",int64
5,consumo_frutas,Em quantos dias da semana o(a) Sr(a) costuma c...,int64
6,consumo_doces,Em quantos dias da semana o(a) Sr(a) costuma c...,int64
7,consumo_substitui_almoco,Em quantos dias da semana o(a) Sr(a) costuma s...,int64
8,percepcao_consumo_sal,Considerando a comida preparada na hora e os a...,int64
9,freq_bebida_alcoolica,Com que frequência o(a) Sr(a) costuma consumir...,int64
